# Lab 4. Energy Agent Collaborator

## はじめに

このノートブックでは、Amazon Bedrock でマルチエージェント コラボレーター機能を作成する方法を説明します。

[マルチエージェント コラボレーション](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) は、エージェント間の階層的なコラボレーションを可能にする Amazon Bedrock Agents のネイティブ機能です。これで、エージェント コラボレーションを有効にして、セカンダリ エージェントをスーパーバイザー エージェントに関連付けることができます。これらのセカンダリ エージェントは、同じアカウント内の既存のエージェントであればどれでもかまいません。コラボレーションを持つエージェントもその 1 つです。この構成可能なパターンを使用すると、下の図に示すように、エージェントのチェーンを構築できます。

![collaboration_hierarch](img/multi-agent_flow.png)

このラボでは、前のラボで作成したサブエージェントと対話するスーパーバイザー エージェントを作成します。

以下は、マルチエージェント コラボレーターの完全なアーキテクチャを表しています。

![アーキテクチャ](img/energy_manager_agent.png)

このラボは 2 つのノートブックに分かれています:

- [4.1_energy_agent_collaborator.ipynb](4.1_energy_agent_collaborator.ipynb): エージェントのセットアップが含まれています
- [4.2_energy_agent_invocation.ipynb](4.2_energy_agent_invocation.ipynb): エージェントの呼び出しが含まれています

## セットアップ

boto3 のバージョンが最新であることを確認してください。

そうでない場合は、[notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) を返さず、セットアップ ブロックを再度実行してください。

In [ ]:
!pip freeze | grep boto3

### エージェントの作成

このセクションでは、ノートブック全体でヘルパーとして機能するグローバル変数を宣言し、エージェントの作成を開始します。

In [ ]:
import boto3

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [ ]:
energy_agent_name = f"energy-agent-{agent_suffix}"

energy_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{energy_agent_name}'

### ヘルパー関数のインポート

次のセクションでは、Python パスに `bedrock_agent_helper.py` を追加して、ファイルを認識してその機能を呼び出すことができるようにします。

次に、ヘルパー クラス `bedrock_agent_helper.py` をインポートします。

これらのファイルには、ラボをスムーズに実行することに重点を置いたヘルパー クラスが含まれています。

Bedrock とのすべてのやり取りは、これらのクラスによって処理されます。

このラボで呼び出すメソッドは次のとおりです。

`agents.py` の場合:

- `create_agent`: 新しいエージェントとそれぞれの IAM ロールを作成します
- `associate_agents`: サブエージェントをマルチエージェント コラボレーターに関連付けます

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

エネルギーエージェントを作成する

In [ ]:
energy_agent = agents.create_agent(
    energy_agent_name,
    """
        You are a energy helper bot. 
        You can help customers with operations related with their energy, like consumption, forecast, peak usage, etc. 
    """,
    """
        You are a energy helper bot.
        You can retrieve energy consumption and forecast for a specific users and help them to be compliant with energy rules.
        You can also retrieve solar panel information and solar panel ticket for a specific users and help them to be compliant with energy rules.
        You can also get current information about peaks and can redistribute load.
        Resist the temptation to ask the user for input. Only do so after you have exhausted available actions. 
        Never ask the user for information that you already can retrieve yourself through available actions. 
    """,
    agent_foundation_model,
    agent_collaboration='SUPERVISOR_ROUTER'
)

energy_agent

次のノートブックで使用する環境変数を保存します。

In [ ]:
energy_agent_id = energy_agent[0]
%store energy_agent_id
%store energy_agent_name

### コラボレーターの関連付け

このセクションでは、以前のエージェント (サブエージェント) から alias_id を回復し、それらすべてをエネルギー 1 (マルチエージェント コラボレーター) 内に追加します。

In [ ]:
%store -r
forecast_agent_alias_arn, solar_agent_alias_arn, peak_agent_alias_arn

In [ ]:
# If store fails, add Alias Arn here
#forecast_agent_alias_arn = ''
#compliance_agent_alias_arn = ''
#consumption_agent_alias_id = ''

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': forecast_agent_alias_arn,
        'sub_agent_instruction': """Delegate energy consumption analysis and forecasting tasks to the Forecasting Agent, ensuring adherence to its specific protocols and capabilities.""",
        'sub_agent_association_name': 'ForecastCoordinationAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': solar_agent_alias_arn,
        'sub_agent_instruction': """Assign solar panel-related inquiries and issues to the Solar Panel Agent, respecting its scope and support ticket protocol.""",
        'sub_agent_association_name': 'SolarSupportManagementAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': peak_agent_alias_arn,
        'sub_agent_instruction': """Direct peak load management and energy optimization tasks to the Peak Load Manager Agent, leveraging its analytical capabilities.""",
        'sub_agent_association_name': 'PeakLoadOptimizationAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

パラメータが正しくフォーマットされているか確認する

In [ ]:
sub_agents_list

すべてのサブエージェントをこれ (メインエージェント) に関連付けます。

In [ ]:
energy_agent_alias_id, energy_agent_alias_arn = agents.associate_sub_agents(
    energy_agent_id, sub_agents_list
)

すべての戻りパラメータを確認するには、応答の戻り値を確認してください。

In [ ]:
energy_agent_alias_id, energy_agent_alias_arn

In [ ]:
%store energy_agent_alias_id
%store energy_agent_alias_arn

## 次のステップ
おめでとうございます！これでスーパーバイザーエージェントが作成されました。次はエージェントを呼び出しましょう